# A Basic Test for the Cytoscape Viewer

In [ ]:
import json
import os
import time
from pathlib import Path

import importnb
from rdflib import Graph as RDFGraph
from rdflib import Literal
from rdflib.namespace import RDF, RDFS, Namespace
from requests_cache import CachedSession

from ipyradiant.visualization import CytoscapeViewer

This notebook is used as part of the `ipyradiant` test suite, where `IPYRADIANT_TESTING`
will be set, which will trigger the automted section below.

In [ ]:
# test graph
EXPECTED_LEN_RDF = 0
EXPECTED_LEN_NX_NODES = 0
EXPECTED_LEN_NX_EDGES = 0

EX = Namespace("https://www.example.com/")
graph = RDFGraph()
graph.namespace_manager.bind("ex", EX)

s1 = EX.subject_1
graph.add((s1, RDF.type, EX.Thing))
graph.add((s1, RDFS.label, Literal("Thing 1")))
EXPECTED_LEN_RDF += 2
EXPECTED_LEN_NX_NODES += 1

s2 = EX.subject_2
graph.add((s2, RDF.type, EX.Thing))
graph.add((s2, RDFS.label, Literal("Thing 2")))
EXPECTED_LEN_RDF += 2
EXPECTED_LEN_NX_NODES += 1

graph.add((s1, EX.relatedTo, s2))
EXPECTED_LEN_RDF += 1
EXPECTED_LEN_NX_EDGES += 1

In [ ]:
def timestamp(msg):
    print(f"[{time.time()}]", msg)

In [ ]:
try:
    timestamp("initiating...")
    cv = CytoscapeViewer()
    timestamp("initiated...")
    assert cv.graph is None
    timestamp("setting rdf graph...")
    assert len(graph) == EXPECTED_LEN_RDF
    cv.graph = graph
    assert len(cv.graph), "Graph failed to populate."
    timestamp("graph set successfully")
    assert len(cv.cytoscape_widget.graph.nodes) > 0, "CytoscapeWidget failed to update."
    assert len(cv.cytoscape_widget.graph.nodes) == EXPECTED_LEN_NX_NODES
    assert len(cv.cytoscape_widget.graph.edges) == EXPECTED_LEN_NX_EDGES
    timestamp("graph converted successfully...")
    timestamp("OK!")
except Exception as err:
    timestamp("ERROR")
    timestamp(err)
    raise Exception(f"{p} failed") from err